In [1]:
# pygame 설치 - 17점수까지
# pip install pygame

SyntaxError: invalid syntax (Temp/ipykernel_18800/204490246.py, line 2)

###### 비행기 게임 Study

1. 2022/03/20 학습 완료 

In [1]:
import pygame 
import random  

"""
====================
게임 초기화 및 화면 너비
====================
"""
pygame.init()
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 800
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT)) 
clock = pygame.time.Clock() 

"""
====================
초기 변수 (변수)
====================
"""
# 이미지 위치 
location_url = 'shoot_rock/'

# 화면 색상 변수
screenColor_Black = (0, 0, 0)
screenColor_Red = (255, 0, 0)
screenColor_Green = (0, 255, 0)
screenColor_Blue = (0, 0, 255)
screenColor_White = (255, 255, 255)
screenColor_Gray = (128, 128, 128)
screenColor_Yellow = (255, 255, 0)

# 폰트 변수 생성 
large_font = pygame.font.SysFont('malgungothic', 72)
small_font = pygame.font.SysFont('malgungothic', 36)

# 이미지 변수 생성 
background_image = pygame.image.load(location_url + 'planet.png')
fighter_image = pygame.image.load(location_url + 'fighter.png')
attack_image = pygame.image.load(location_url + 'missile1.png')
explosion_image = pygame.image.load(location_url + 'explosion.png')

# 장애물
wall_level_1_image = pygame.image.load(location_url + 'wall_level_1.png')
wall_level_2_image = pygame.image.load(location_url + 'wall_level_2.png')

# 음악 생성
pygame.mixer.init()
pygame.mixer.music.load(location_url + 'JLD_-_Lethal_Xcess_Level_4.mp3')
pygame.mixer.music.play(-1) # 무한 반복
attack_sound = pygame.mixer.Sound(location_url + 'missile.wav')
explosion_sound = pygame.mixer.Sound(location_url + 'explosion.wav')
game_over_sound = pygame.mixer.Sound(location_url + 'game_over.wav')

# 변수 및 배열 선언
fighter = fighter_image.get_rect(centerx=SCREEN_WIDTH // 2, bottom=SCREEN_HEIGHT)
rocks = []
double_rocks = []
attacks = []
missiles = []
score = 0
missed = 0
gameOver = False
random_number = random.randint(3, 9)
double_attack_count = 0


# 바위 생성
for i in range(3):
    rock = wall_level_1_image.get_rect(left=random.randint(0, SCREEN_WIDTH - wall_level_1_image.get_width()), top= -100)
    dy = random.randint(3, 9)
    rocks.append((rock, dy))


# 더블 바위 생성 
for i in range(1):
    double_rock = wall_level_2_image.get_rect(left=random.randint(0, SCREEN_WIDTH - wall_level_2_image.get_width()), top= -100)
    dy = random.randint(3, 9)
    double_rocks.append((double_rock, dy))
    
    
    
# pygame.key.set_repeat(1, 1)

"""
====================
게임 루프 로직 (실제 로직)
====================
"""
while True: 
    screen.blit(background_image, (0, 0))

    #변수 업데이트
    event = pygame.event.poll() #이벤트 처리
    if event.type == pygame.QUIT:
        break
    elif event.type == pygame.KEYDOWN:
        if event.key == pygame.K_SPACE:
            attack = pygame.Rect(attack_image.get_rect(centerx=fighter.centerx, top=fighter.top))
            attacks.append(attack)
            attack_sound.play()
        
    pressed = pygame.key.get_pressed()
    if pressed[pygame.K_LEFT] and not gameOver:
        fighter.left -= 5
    elif pressed[pygame.K_RIGHT] and not gameOver:
        fighter.left += 5
    
    if not gameOver:
        # 장애물 로직
        for rock, dy in rocks:
            rock.top += dy
            if rock.top > SCREEN_HEIGHT:
                rocks.remove((rock, dy))
                rock = wall_level_1_image.get_rect(left=random.randint(0, SCREEN_WIDTH - wall_level_1_image.get_width()), top= -100)
                dy = random.randint(3, 9)
                rocks.append((rock, dy))
                missed += 1
        
        # 더블 장애물 로직 
        for double_rock, dy in double_rocks:
            double_rock.top += dy
            if double_rock.top > SCREEN_HEIGHT:
                double_rocks.remove((double_rock, dy))
                double_rock = wall_level_2_image.get_rect(left=random.randint(0, SCREEN_WIDTH - wall_level_2_image.get_width()), top= -100)
                dy = random.randint(3, 9)
                double_rocks.append((double_rock, dy))
                missed += 1
        
        
        # 종료 체크 로직
        if missed >= 3:
            gameOver = True
            pygame.mixer.music.stop()
            game_over_sound.play()
            

        # 공격 로직
        for attack in attacks:
            attack.top -= 20        # 공격속도
            if attack.top < 0:
                attacks.remove(attack)


        # 비행기 이동 로직 
        if fighter.left < 0:
            fighter.left = 0
        elif fighter.right > SCREEN_WIDTH:
            fighter.right = SCREEN_WIDTH

        # 장애물 격파 로직 
        for rock, dy in rocks:
            for attack in attacks:
                if attack.colliderect(rock):
                    rocks.remove((rock, dy))
                    attacks.remove(attack)
                    screen.blit(explosion_image, (rock.left, rock.top))
                    rock = wall_level_1_image.get_rect(left=random.randint(0, SCREEN_WIDTH - wall_level_1_image.get_width()), top= -100)
                    dy = random.randint(3, 9)
                    rocks.append((rock, dy))
                    score += 1
                    explosion_sound.play()

        # 더블 장애물 로직 
        for double_rock, dy in double_rocks:
            for attack in attacks:
                if attack.colliderect(double_rock):
                    attacks.remove(attack)
                    double_attack_count += 1
                
                    # 2번 공격했을 경우 
                    if double_attack_count == 2:    
                        double_rocks.remove((double_rock, dy))
#                         attacks.remove(attack)
                        screen.blit(explosion_image, (double_rock.left, double_rock.top))
                        double_rock = wall_level_2_image.get_rect(left=random.randint(0, SCREEN_WIDTH - wall_level_2_image.get_width()), top= -100)
                        dy = random.randint(3, 9)
                        double_rocks.append((double_rock, dy))
                        score += 1
                        double_attack_count = 0
                        explosion_sound.play()
                       
        for rock, dy in rocks:
            if rock.colliderect(fighter):
                gameOver = True
                pygame.mixer.music.stop()
                game_over_sound.play()
    
        for double_rock, dy in double_rocks:
            if double_rock.colliderect(fighter):
                gameOver = True
                pygame.mixer.music.stop()
                game_over_sound.play()
   
    for rock, dy in rocks:
        screen.blit(wall_level_1_image, rock)

    for double_rock, dy in double_rocks:
        screen.blit(wall_level_2_image, double_rock)
        
    for attack in attacks:
        screen.blit(attack_image, attack)

    screen.blit(fighter_image, fighter)

    score_image = small_font.render('점수 {}'.format(score), True, screenColor_Yellow)
    screen.blit(score_image, (10, 10))

    missed_image = small_font.render('놓친 운석 {}'.format(missed),  True, screenColor_Yellow)
    screen.blit(missed_image, missed_image.get_rect(right=SCREEN_WIDTH - 10, top=10))

    if gameOver:
        game_over_image = large_font.render('Game Over', True, screenColor_Red)
        screen.blit(game_over_image, game_over_image.get_rect(centerx=SCREEN_WIDTH // 2, centery= SCREEN_HEIGHT // 2))

    pygame.display.update() 
    # FPS 프레임
    clock.tick(30) 

    
pygame.quit() 

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
1
2
1
2
1
2
1
2


4